## DPK 반납 정보를 가공하여 데이타프레임(df) 으로 만듬

In [1]:
from tool import *

In [2]:
df = pd.read_excel(get_filename())
df['DPK_ID'] = df['DPK_ID'].astype(str)
df['MS_DateTime'] = df['MS_DateTime'].astype(str)
df['MS_DateTime'] = pd.to_datetime(df['MS_DateTime'])
df

,DPK_ID,CUST_NAME,CUST_DPK_PN,QCI_DPK_PN,MS_DPK_PN,Status,UpdateDateTime,CUST_DateTime,MS_DateTime,Unnamed: 9
0,3305441319100,LG,SAE30320201,AZDPKW10068,FQC-08797,RTN,20230207111305,20221013151341,2022-10-13 15:13:41,Return success already？
1,3305441319097,LG,SAE30320201,AZDPKW10068,FQC-08797,RTN,20230129170052,20221013151341,2022-10-13 15:13:41,Return success already？
2,3305441319096,LG,SAE30320201,AZDPKW10068,FQC-08797,RTN,20230207150152,20221013151341,2022-10-13 15:13:41,Return success already？
3,3305441319099,LG,SAE30320201,AZDPKW10068,FQC-08797,RTN,20230207111850,20221013151341,2022-10-13 15:13:41,Return success already？
4,3305439630654,LG,SAE30320201,AZDPKW10068,FQC-08797,RTN,20230207150131,20220729082351,2022-07-29 08:23:51,Paul advice return fail
...,...,...,...,...,...,...,...,...,...,...
425,3422097686092,LG,SAE30761301,AZDPKW11196,KW9-00609,RTN,20221217134203,20221122102351,2022-11-22 10:23:51,NaN
426,3422097686083,LG,SAE30761301,AZDPKW11196,KW9-00609,RTN,20221217174211,20221122102351,2022-11-22 10:23:51,NaN
427,3422097686114,LG,SAE30761301,AZDPKW11196,KW9-00609,RTN,20221217134203,20221122102351,2022-11-22 10:23:51,NaN
428,3422097686143,LG,SAE30761301,AZDPKW11196,KW9-00609,RTN,20221217134203,20221122102351,2022-11-22 10:23:51,NaN


## 1. DPK 반납 요청 리스트의 데이타 이상 여부 확인

In [38]:
# DPK ID 반납 요청 개수 확인
row_qty = len(df['DPK_ID'])
return_qty = len(df['DPK_ID'].unique())
str_len_check = df['DPK_ID'].str.len()
print(f'1. DPK ID의 행의 개수: {row_qty}')
print(f'2. DPK ID의 반납 수량: {return_qty}')
print(f'3. DPK 자리수(13자리) 이상한 것의 개수 : {(str_len_check != 13).sum()}') # 요청 DPK의 자리수가 13자리로 되어 있는지 확인
print(f'4. DPK P/N 자리수(9자리) 이상한 것의 개수: {(df["MS_DPK_PN"].astype(str).str.len() != 9 ).sum()}') # DPK ID 의 자리수가 9자리로 되어 있는지 확인

1. DPK ID의 행의 개수: 430
2. DPK ID의 반납 수량: 430
3. DPK 자리수(13자리) 이상한 것의 개수 : 0
4. DPK P/N 자리수(9자리) 이상한 것의 개수: 0


In [4]:
df['DPK Name'] = df['MS_DPK_PN'].map(dpk_map) # DPK Name 컬럼에 DPK PN에 맞는 Name matching

In [40]:
print(df.groupby(['MS_DPK_PN', 'DPK Name']).size()) # DPK 종류별 수량 확인
print('총 수량 :', df.groupby(['MS_DPK_PN', 'DPK Name']).size().sum()) # DPK 종류별 수량 확인

MS_DPK_PN  DPK Name            
CSQ-00009  Windows 11 Pro Entry     33
FQC-08797  Win 10 Pro                7
FQC-10428  Win 11 Pro               59
KK3-00027  Win 11 Home Plus        193
KW9-00609  Win 11 Home             138
dtype: int64
총 수량 : 430


## PKID가 PC OA의 DPK Fullfillment DB에 기 포함되어 있는지 확인

In [10]:
with open('D:/Data/DPK Fullfillment DB.bin', 'rb') as f:
    ff_db = pickle.load(f)

In [11]:
dpk_dup = pd.DataFrame(columns=df.columns)
dpk_dup

,DPK_ID,CUST_NAME,CUST_DPK_PN,QCI_DPK_PN,MS_DPK_PN,Status,UpdateDateTime,CUST_DateTime,MS_DateTime,Unnamed: 9,DPK Name


In [12]:
for row in df.iterrows():
    if row[1]['DPK_ID'] in ff_db['PKID'].tolist():
        dpk_dup = dpk_dup.append(row[1])
dpk_dup

,DPK_ID,CUST_NAME,CUST_DPK_PN,QCI_DPK_PN,MS_DPK_PN,Status,UpdateDateTime,CUST_DateTime,MS_DateTime,Unnamed: 9,DPK Name
0,3305441319100,LG,SAE30320201,AZDPKW10068,FQC-08797,RTN,20230207111305,20221013151341,2022-10-13 15:13:41,Return success already？,Win 10 Pro
1,3305441319097,LG,SAE30320201,AZDPKW10068,FQC-08797,RTN,20230129170052,20221013151341,2022-10-13 15:13:41,Return success already？,Win 10 Pro
2,3305441319096,LG,SAE30320201,AZDPKW10068,FQC-08797,RTN,20230207150152,20221013151341,2022-10-13 15:13:41,Return success already？,Win 10 Pro
3,3305441319099,LG,SAE30320201,AZDPKW10068,FQC-08797,RTN,20230207111850,20221013151341,2022-10-13 15:13:41,Return success already？,Win 10 Pro
4,3305439630654,LG,SAE30320201,AZDPKW10068,FQC-08797,RTN,20230207150131,20220729082351,2022-07-29 08:23:51,Paul advice return fail,Win 10 Pro
...,...,...,...,...,...,...,...,...,...,...,...
425,3422097686092,LG,SAE30761301,AZDPKW11196,KW9-00609,RTN,20221217134203,20221122102351,2022-11-22 10:23:51,NaN,Win 11 Home
426,3422097686083,LG,SAE30761301,AZDPKW11196,KW9-00609,RTN,20221217174211,20221122102351,2022-11-22 10:23:51,NaN,Win 11 Home
427,3422097686114,LG,SAE30761301,AZDPKW11196,KW9-00609,RTN,20221217134203,20221122102351,2022-11-22 10:23:51,NaN,Win 11 Home
428,3422097686143,LG,SAE30761301,AZDPKW11196,KW9-00609,RTN,20221217134203,20221122102351,2022-11-22 10:23:51,NaN,Win 11 Home


In [13]:
dpk_dup.shape

(430, 11)

## 1단계 : DPK return 전 반납하려는 DPK 의 Fullfillment 정보를 등록하기 위한 format 생성

In [88]:
# fullfillment 정보 등록하는 format의 컬럼명를 설정
col = ['Site', 'MS P/No.', 'PKID', 'FulfilledDateUTC', 'OEM PO Number', 'Date of Update', 'Remark']

In [89]:
f1 = datetime.date.today().strftime('%y%m%d')
return_PO_num = f'QT{f1}_01'

df1 = df[['MS_DPK_PN', 'DPK_ID', 'MS_DateTime' ]]
df1.insert(0, 'Site', 'QT')
df1['OEM PO Number'] = return_PO_num
df1['Date Of Update'] = None
df1['Remark'] = None
df1

C:\Users\paul76.lee\AppData\Local\Temp/ipykernel_18360/2853372322.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['OEM PO Number'] = return_PO_num
C:\Users\paul76.lee\AppData\Local\Temp/ipykernel_18360/2853372322.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Date Of Update'] = None
C:\Users\paul76.lee\AppData\Local\Temp/ipykernel_18360/2853372322.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

,Site,MS_DPK_PN,DPK_ID,MS_DateTime,OEM PO Number,Date Of Update,Remark
0,QT,FQC-08797,3305441319100,2022-10-13 15:13:41,QT230407_01,None,None
1,QT,FQC-08797,3305441319097,2022-10-13 15:13:41,QT230407_01,None,None
2,QT,FQC-08797,3305441319096,2022-10-13 15:13:41,QT230407_01,None,None
3,QT,FQC-08797,3305441319099,2022-10-13 15:13:41,QT230407_01,None,None
4,QT,FQC-08797,3305439630654,2022-07-29 08:23:51,QT230407_01,None,None
...,...,...,...,...,...,...,...
425,QT,KW9-00609,3422097686092,2022-11-22 10:23:51,QT230407_01,None,None
426,QT,KW9-00609,3422097686083,2022-11-22 10:23:51,QT230407_01,None,None
427,QT,KW9-00609,3422097686114,2022-11-22 10:23:51,QT230407_01,None,None
428,QT,KW9-00609,3422097686143,2022-11-22 10:23:51,QT230407_01,None,None


In [90]:
df1.columns = ['Site', 'MS P/No.', 'PKID', 'FulfilledDateUTC', 'OEM PO Number', 'Date of Update', 'Remark']

In [91]:
df1

,Site,MS P/No.,PKID,FulfilledDateUTC,OEM PO Number,Date of Update,Remark
0,QT,FQC-08797,3305441319100,2022-10-13 15:13:41,QT230407_01,None,None
1,QT,FQC-08797,3305441319097,2022-10-13 15:13:41,QT230407_01,None,None
2,QT,FQC-08797,3305441319096,2022-10-13 15:13:41,QT230407_01,None,None
3,QT,FQC-08797,3305441319099,2022-10-13 15:13:41,QT230407_01,None,None
4,QT,FQC-08797,3305439630654,2022-07-29 08:23:51,QT230407_01,None,None
...,...,...,...,...,...,...,...
425,QT,KW9-00609,3422097686092,2022-11-22 10:23:51,QT230407_01,None,None
426,QT,KW9-00609,3422097686083,2022-11-22 10:23:51,QT230407_01,None,None
427,QT,KW9-00609,3422097686114,2022-11-22 10:23:51,QT230407_01,None,None
428,QT,KW9-00609,3422097686143,2022-11-22 10:23:51,QT230407_01,None,None


In [93]:
df2 = df1[~df1['PKID'].isin(dpk_dup['DPK_ID'])] # 기 등록된 PKID는 제외함

In [94]:
ff_qty = return_qty - dpk_dup.shape[0] # fullfillment 내역으로 등록할 수량

In [95]:
df2.to_excel(f'D:/Python result/DPK Return/{f1}_Quanta_DPK_Fullfillment_register_{ff_qty}.xlsx', index = False)

## 2단계 : DPK 반납을 위한 DPK와 S/N의 매칭 정보(CBR)를 업로드
 * 현재 PCOA의 CBR 처리 방식은 PKID 가 이전에 등록이 되었는지 check 하여 중복된 PKID는 업로드시 에러를 발생시킴(SET S/N 은 중복여부를 check 하지 않음) - 23.4.17 확인(담당자 : 최재혁 책임)

In [101]:
df_cbr = pd.read_excel(get_filename(), sheet_name='430pcs PKID&SN mapping')
df_cbr['DPK_ID'] = df_cbr['DPK_ID'].astype(str)
df_cbr['QCI_SN'] = df_cbr['QCI_SN'].str.strip()

In [102]:
row_qty = len(df_cbr['DPK_ID'])
dpk_unique_qty = len(df_cbr['DPK_ID'].unique().tolist())
sn_unique_qty = len(df_cbr['QCI_SN'].unique().tolist())
str_len_check1 = df_cbr['DPK_ID'].str.len()
str_len_check2 = df_cbr['QCI_SN'].str.len()

print(f'1. DPK ID의 행의 개수: {row_qty}')
print(f'2. DPK ID의 수량: {dpk_unique_qty}')
print(f'3. SET S/N의 수량: {sn_unique_qty}')
print(f'4. DPK 자리수(13자리) 이상한 것의 개수 : {(str_len_check1 != 13).sum()}') # 요청 DPK의 자리수가 13자리로 되어 있는지 확인
print(f'5. Set S/N 자리수(13자리) 이상한 것의 개수 : {(str_len_check2 != 13).sum()}') # SET S/N의 자리수가 13자리로 되어 있는지 확인

1. DPK ID의 행의 개수: 432
2. DPK ID의 수량: 430
3. SET S/N의 수량: 431
4. DPK 자리수(13자리) 이상한 것의 개수 : 0
5. Set S/N 자리수(13자리) 이상한 것의 개수 : 0


In [103]:
df_cbr = df_cbr.drop_duplicates(subset=['DPK_ID'], keep='last') # 중복된 DPK ID를 삭제하는데 가장 마지막 행의 DPK ID만 남기고 삭제함

In [104]:
set(df['DPK_ID'].unique()) - set(df_cbr['DPK_ID'].unique()) # 2개의 데이타(반납할 DPK 리스트와 CBR 등록할 DPK 리스트)의 DPK 값이 완전히 일치하는 지 확인

set()

In [105]:
len(df_cbr['DPK_ID'].unique()) # 등록할 CBR 데이터의 unique 한 DPK ID 개수를 확인

430

In [106]:
with open('D:/Data/DPK CBR DB.bin', 'rb') as f:
    cbr_db = pickle.load(f)

df_cbr1 = df_cbr[~df_cbr['DPK_ID'].isin(cbr_db['PKID'])] # 기존에 CBR로 등록된 DPK ID 제외한 리스트로 만듬

In [107]:
df_cbr1.shape

(430, 3)

In [90]:
df_cbr1 = df_cbr1[['DPK_ID', 'QCI_SN']].merge(df[['DPK_ID', 'MS_DPK_PN', 'MS_DateTime']])
df_cbr1['SITE'] = 'QT' # site 정보 추가
df_cbr1.columns = ['PKID', 'SET serial No.', 'MS P/No.', 'MSReceivedDateUTC ', 'SITE'] # 컬럼명 변경
df_cbr1['Mac Address'] = None # Mac address 컬럼 추가
df_cbr1['Remark'] = None # Remark 컬럼 추가
df_cbr1 = df_cbr1[['SITE', 'SET serial No.', 'MS P/No.', 'PKID', 'MSReceivedDateUTC ', 'Mac Address', 'Remark']] # 컬럼 순서를 PC OA의 CBR 등록 포맷에 맞게 변경

In [97]:
f1 = datetime.date.today().strftime('%y%m%d')
df_cbr1.to_excel(f'D:/Python result/DPK Return/{f1}_Quanta CBR register before DPK return_{df_cbr1.shape[0]}.xlsx', index = False)

## 3단계 : DPK RMA Return 포맷 작성

In [172]:
df_rma_return = df[['DPK_ID', 'MS_DPK_PN']].merge(df_cbr[['DPK_ID', 'QCI_SN']]) # Quanta가 요청하는 리턴 리스트(첫번째 시트)와 DPK와 S/N 맵핑 리스트를 merge하여 하나로 만듬
df_rma_return

,DPK_ID,MS_DPK_PN,QCI_SN
0,3305441319100,FQC-08797,112QCVU564071
1,3305441319097,FQC-08797,201QCWC557073
2,3305441319096,FQC-08797,112QCAS562552
3,3305441319099,FQC-08797,112QCWC567233
4,3305439630654,FQC-08797,202QCBD559365
...,...,...,...
425,3422097686092,KW9-00609,212QCAS558840
426,3422097686083,KW9-00609,212QCSF558038
427,3422097686114,KW9-00609,212QCHE558801
428,3422097686143,KW9-00609,212QCRN558846


In [173]:
df_rma_return['Site'] = 'QT'
df_rma_return['Return Type'] = 'ZOC'
df_rma_return['No Credit'] = 'false'
df_rma_return[['ReturnDateUTC', 'Remark']] = None

df_rma_return.columns = ['PKID', 'MS P/No.', 'SET serial No.', 'Site', 'Return Type', 'No Credit', 'ReturnDateUTC', 'Remark']
df_rma_return = df_rma_return[['Site', 'SET serial No.', 'MS P/No.', 'PKID', 'Return Type', 'No Credit', 'ReturnDateUTC', 'Remark']]

In [174]:
df_rma_return

,Site,SET serial No.,MS P/No.,PKID,Return Type,No Credit,ReturnDateUTC,Remark
0,QT,112QCVU564071,FQC-08797,3305441319100,ZOC,false,None,None
1,QT,201QCWC557073,FQC-08797,3305441319097,ZOC,false,None,None
2,QT,112QCAS562552,FQC-08797,3305441319096,ZOC,false,None,None
3,QT,112QCWC567233,FQC-08797,3305441319099,ZOC,false,None,None
4,QT,202QCBD559365,FQC-08797,3305439630654,ZOC,false,None,None
...,...,...,...,...,...,...,...,...
425,QT,212QCAS558840,KW9-00609,3422097686092,ZOC,false,None,None
426,QT,212QCSF558038,KW9-00609,3422097686083,ZOC,false,None,None
427,QT,212QCHE558801,KW9-00609,3422097686114,ZOC,false,None,None
428,QT,212QCRN558846,KW9-00609,3422097686143,ZOC,false,None,None


In [175]:
with open('D:/Data/DPK return db.bin', 'rb') as f: # DPK return DB 를 불러옴
    rt_db = pickle.load(f)

In [178]:
pkid_dup = df_rma_return[(df_rma_return['PKID'].isin(rt_db['PKID']))][['PKID', 'Remark']]  # DPK return DB에 등록했던 PKID 리스트(중복임)
pkid_dup['Remark'] = 'This DPK ID is already returned before by DPK return process'

In [180]:
df_rma_return = df_rma_return[~(df_rma_return['PKID'].isin(rt_db['PKID']))] # DPK return DB에 등록 이력이 없는 DPK만을 재구성

In [181]:
df_rma_return.shape

(424, 8)

In [182]:
with open('D:/Data/DPK RMA Return DB.bin', 'rb') as f: # DPK RMA return DB 를 불러옴
    rma_db = pickle.load(f)

In [186]:
pkid_dup = pd.concat([pkid_dup, df_rma_return[df_rma_return['PKID'].isin(rma_db['PKID'])][['PKID', 'Remark']]]).reset_index(drop=True) # DPK RMA return DB에 등록했던 PKID 리스트(중복임)
pkid_dup.loc[pkid_dup['Remark'].isnull(), 'Remark'] = 'This DPK ID is already returned before by DPK RMA return process'

In [187]:
df_rma_return = df_rma_return[~(df_rma_return['PKID'].isin(rma_db['PKID']))] # RMA 리스트 중 기존에 RMA return 이력이 있는 PKID는 제외하여 다시 구성
df_rma_return.shape

(376, 8)

In [171]:
f1 = datetime.date.today().strftime('%y%m%d')
df_rma_return.to_excel(f'D:/Python result/DPK Return/{f1}_quanta_DPK RMA_return_{df_rma_return.shape[0]}.xlsx', index = False)

In [190]:
pkid_dup.to_clipboard()

In [192]:
pkid_dup['PKID'][0]

'3305441319100'